# RAG(Retrieval Augmented Generation)
- [RAG](https://python.langchain.com/v0.1/docs/modules/data_connection/)은 *Retrieval Augmented Generation*의 약자로, **검색 기반 생성 기법**을 의미한다. 이 기법은 LLM이 특정 문서에 기반하여 보다 정확하고 신뢰할 수 있는 답변을 생성할 수 있도록 돕는다.     
- 사용자의 질문에 대해 자체적으로 구축한 데이터베이스(DB)나 외부 데이터베이스에서 질문과 관련된 문서를 검색하고, 이를 질문과 함께 LLM에 전달한다.
- LLM은 같이 전달된 문서를 바탕으로 질문에 대한 답변을 생성한다. 
- 이를 통해 LLM이 학습하지 않은 내용도 다룰 수 있으며, 잘못된 정보를 생성하는 환각 현상(*hallucination*)을 줄일 수 있다.

## RAG와 파인튜닝(Fine Tuning) 비교

### 파인튜닝(Fine Tuning)

- **정의**: 사전 학습(pre-trained)된 LLM에 특정 도메인의 데이터를 추가로 학습시켜 해당 도메인에 특화된 맞춤형 모델로 만드는 방식이다.
- **장점**
  - 특정 도메인에 최적화되어 높은 정확도와 성능을 낼 수 있다.
- **단점**
  - 모델 재학습에 많은 시간과 자원이 필요하다.
  - 새로운 정보가 반영되지 않으며, 이를 위해서는 다시 학습해야 한다.

### RAG

- **정의**: 모델을 다시 학습시키지 않고, 외부 지식 기반에서 정보를 검색하여 실시간으로 답변에 활용하는 방식이다.
- **장점**
  - 최신 정보를 쉽게 반영할 수 있다.
  - 모델을 수정하지 않아도 되므로 효율적이다.
- **단점**
  - 검색된 문서의 품질에 따라 답변의 정확성이 달라질 수 있다.
  - 검색 시스템 구축이 필요하다.

## 정리

| 항목       | 파인튜닝 | RAG |
| -------- | ---- | --- |
| 도메인 최적화  | 가능   | 제한적 |
| 최신 정보 반영 | 불가능  | 가능  |
| 구현 난이도   | 높음   | 보통  |
| 유연성      | 낮음   | 높음  |

- LLM은 학습 당시의 데이터만을 기반으로 작동하므로 최신 정보나 기업 내부 자료와 같은 특정한 지식 기반에 접근할 수 없다.
- 파인튜닝은 시간과 비용이 많이 들고 유지보수가 어렵다.
-	반면, RAG는 기존 LLM을 변경하지 않고도 외부 문서를 통해 그 한계를 보완할 수 있다.
- RAG는 특히 빠르게 변화하는 정보를 다루는 분야(예: 기술 지원, 뉴스, 법률 등)에서 유용하게 활용된다. 반면, 정적인 정보에 대해 높은 정확도가 필요한 경우에는 파인튜닝이 효과적이다.


## RAG 작동 단계
- 크게 "**정보 저장(인덱싱)**", "**검색**, **생성**"의 단계로 나눌 수 있다.
  
### 1. 정보 저장(인덱싱)
RAG는 사전에 정보를 가공하여 **벡터 데이터베이스**(Vector 저장소)에 저장해 두고, 나중에 검색할 수 있도록 준비한다. 이 단계는 다음과 같은 과정으로 이루어진다.

1. **Load (불러오기)**
   - 답변시 참조할 사전 정보를 가진 데이터들을 불러온다.
2. **Split/Chunking (문서 분할)**
   - 긴 텍스트를 일정한 길이의 작은 덩어리(*chunk*)로 나눈다.
   - 이렇게 해야 검색과 생성의 정확도를 높일 수 있다.
3. **Embedding (임베딩)**
   - 각 텍스트 조각을 **임베딩 벡터**로 변환한다.
   - 임베딩 벡터는 그 문서의 의미를 벡터화 한 것으로 질문과 유사한 문서를 찾을 때 인덱스로 사용된다.
4. **Store (저장)**
   - 임베딩된 벡터를 **벡터 데이터베이스**(벡터 저장소)에 저장한다.
   - 벡터 데이터베이스는 유사한 질문이나 문장을 빠르게 찾을 수 있도록 특화된 데이터 저장소이다.
   
![rag](figures/rag1.png)

### 2. 검색, 생성

사용자가 질문을 하면 다음과 같은 절차로 답변이 생성된다.
1. **Retrieve (검색)**
   - 사용자의 질문을 임베딩한 후, 이 질문 벡터와 유사한 context 벡터를 벡터 데이터베이스에서 검색하여 찾는다.
2. **Query (질의 생성)**
   - 벡터 데이터베이스에서 검색된 문서 조각과 사용자의 질문을 함께 **프롬프트**(prompt)로 구성하여 LLM에 전달한다.
3. **Generation (응답 생성)**
   - LLM은 받은 프롬프트에 대한 응답을 생성한다.
   
- **RAG 흐름**
  
![Retrieve and Generation](figures/rag2.png)


# Document Loader
- LLM에게 질의할 때 같이 제공할 Data들을 저장하기 위해 먼저 읽어들인다.(Load)
- 데이터 Resouce는 다양하다.
    - 데이터를 로드(load)하는 방식은 저장된 위치와 형식에 따라 다양하다. 
      - 로컬 컴퓨터(Local Computer)에 저장된 문서
        - 예: CSV, Excel, JSON, TXT 파일 등
      - 데이터베이스(Database)에 저장된 데이터셋
      - 인터넷에 존재하는 데이터
        - 예: 웹에 공개된 API, 웹 페이지에 있는 데이터, 클라우드 스토리지에 저장된 파일 등

![rag_load](figures/rag_load.png)

- 다양한 문서 형식(format)에 맞춰 읽어오는 다양한 **document loader** 들을 Langchain에서 지원한다.
    - 다양한 Resource들로 부터 데이터를 읽기 위해서는 다양한 라이브러리를 이용해 서로 다른 방법으로 읽어야 한다.
    - Langchain은 데이터를 읽는 다양한 방식의 코드를 하나의 interface로 사용 할 수 있도록 지원한다.
        - https://python.langchain.com/docs/how_to/#document-loaders
    - 다양한 3rd party library(ppt, github 등등 다양한 3rd party lib도 있음. )들과 연동해 다양한 Resource로 부터 데이터를 Loading 할 수 있다.
        - https://python.langchain.com/docs/integrations/document_loaders/
- **모든 document loader는 기본적으로 동일한 interface(사용법)로 호출할 수있다.**
- **반환타입**
    - **list[Document]**
    - Load 한 문서는 Document객체에 정보들을 넣는다. 여러 문서를 읽을 수 있기 대문에 list에 묶어서 반환한다.
        - **Document 속성**
            - page_content: 문서의 내용
            - metadata(option): 문서에 대한 메타데이터(정보)를 dict 형태로 저장한다. 
            - id(option): 문서의 고유 id
     
- **주의**
    - Langchain을 이용해 RAG를 구현할 때 **꼭 Langchain의 DocumentLoader를 사용해야 하는 것은 아니다.**
    - DocumentLoader는 데이터를 읽어오는 것을 도와주는 라이브러리일 뿐이다. 다른 라이브러리를 이용해서 읽어 들여도 상관없다. 

## 주요 Document Loader

### Text file
- TextLoader 이용

### PDF
- PyPDF, Pymupdf 등 다양한 PDF 문서를 읽어들이는 파이썬의  3rd party library들을 이용해 pdf 문서를 Load 한다.
    - https://python.langchain.com/docs/integrations/document_loaders/#pdfs
- 각 PDF Loader 특징
    -  PyMuPDF
        -   텍스트 뿐 아니라 이미지, 주석등의 정보를 추출하는데 성능이 좋다.
        -   PyMuPDF 라이브러리 기반
    - PyPDFLoader
        - 텍스트를 빠르게 추출 할 수있다.
        - PyPDF2 라이브러리 기반. 경량 라이브러리로 빠르고 큰 파일도 효율적으로 처리한다.
    - PDFPlumberLoader
        - 표와 같은 복잡한 구조의 데이터 처리하는데 강력한 성능을 보여준다. 텍스트, 이미지, 표 등을 모두 추출할 수 있다. 
        - PDFPlumber 라이브러리 기반
- 설치 패키지
    - DocumentLoader와 연동하는 라이브러리들을 설치 해야 한다.
    - `pip install pypdf -qU`
    - `pip install pymupdf -qU`
    - `pip install pdfplumber -qU`

### Web

- WebBaseLoader 이용
  - 입력받은 URL의 웹 문서를 읽어 문서로 로드한다. 웹 크롤링작업 없이 웹상의 문서를 가져올 수있다.
  - 내부적으로 BeautifulSoup을 이용해 웹문서를 parsing한다.
- https://python.langchain.com/docs/how_to/document_loader_web/

### ArxivLoader
- https://github.com/lukasschwab/arxiv.py
- [arXiv-아카이브](https://arxiv.org/) 는 미국 코렐대학에서 운영하는 **무료 논문 저장소**로, 물리학, 수학, 컴퓨터 과학, 생물학, 금융, 경제 등 **과학, 금융 분야의 논문**들을 공유한다.
- `ArxivLoader` 를 사용해 원하는 주제의 논문들을 arXiv에서 가져와 load할 수 있다.
- **arXiv API**를 사용해 논문을 가져올 수 있다.
  - https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.arxiv.ArxivLoader.html
- 설치
  - `pip install langchain-community -qU`
  - `pip install arxiv -qU`



### Docling
- IBM Research에서 개발한 오픈소스 문서처리 도구로 다양한 종류의 문서를 구조화된 데이터로 변환해 생성형 AI에서 활용할 수있도록 지원한다.
- **주요기능**
  - PDF, DOCX, PPTX, XLSX, HTML, 이미지 등 여러 형식을 지원
  - PDF의 **페이지 레이아웃, 읽기 순서, 표 구조, 코드, 수식** 등을 분석하여 정확하게 읽어들인다.
  - OCR을 지원하여 스캔된 PDF나 이미지에서 텍스트를 추출할 수있다.
  - 읽어들인 내용을 markdown, html, json등 다양한 형식으로 출력해준다.
- 설치 : `pip install langchain-docling ipywidgets -qU` 
- 참조
  - docling 사이트: https://github.com/docling-project/docling
  - 랭체인-docling https://python.langchain.com/docs/integrations/document_loaders/docling/

### UnstructuredLoader
- 다양한 비정형 문서들을 읽어 오는 Unstrctured 를 사용해, 다양한 형식의 문서들을 load 해 RAG, 모델 파인튜닝에 적용할 수있게 한다.
  - 지원 파일 형식: "csv", "doc", "docx", "epub", "image", "md", "msg", "odt", "org", "pdf", "ppt", "pptx", "rtf", "rst", "tsv", "xlsx"
- **다양한 형식의 파일로 부터 text를 로딩**해야 할 경우 유용하다. 
- Local에 library를 설치해서 사용하거나,  Unstructured 가 제공하는 API service를 사용할 수 있다.
  - https://docs.unstructured.io
- 텍스트 파일, PDF, 이미지, HTML, XML, ms-office(word, ppt), epub 등 다양한 비정형 데이터 파일을 처리할 수 있다.
  - 설치, 지원 문서: https://docs.unstructured.io/open-source/installation/full-installation
  - Langchain 문서: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file

> - UnstructuredLoader PDF Load 시 Document 분할 기준
>     -  문서의 구조와 콘텐츠를 기반으로 텍스트를 분할해 Document에 넣는다.
>     -  분할 기준
>        - 헤더(Header): 문서의 제목이나 섹션 제목 등
>        - 본문 텍스트(NarrativeText): 일반적인 문단이나 설명문
>        - 표(Table): 데이터가 표 형식으로 구성된 부분
>        - 리스트(List): 순서가 있거나 없는 목록
>        - 이미지(Image): 사진이나 그래픽 요소

#### 설치할 프로그램
- poppler
  - pdf 파일을 text로 변환하기 위해 필요한 프로그램
  - windows: https://github.com/oschwartz10612/poppler-windows/releases/ 에서 최신 버전 다운로드 후 압축 풀어서 설치.
    - 환경변수 Path에 "설치경로\Library\bin" 을 추가. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install poppler`
  - Linux: `sudo apt-get install poppler-utils`
- tesseract-ocr
  - OCR 라이브러리로 pdf 이미지를 text로 변환하기 위해 필요한 프로그램 
  - windows: https://github.com/UB-Mannheim/tesseract/wiki 에서 다운받아 설치. 
    - 환경변수 Path에 설치 경로("C:\Program Files\Tesseract-OCR") 추가 한다. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install tesseract`
  - linux(unbuntu): `sudo apt install tesseract-ocr`
- 설치 할 패키지
  - **libmagic 설치**
      - windows: `pip install python-magic-bin -qU`
      - macOS: `brew install libmagic`
      - linux(ubuntu): `sudo apt-get install libmagic-dev`
  - `pip install "unstructured[pdf]" -qU`
      - 문서 형식별로 sub module을 설치한다. (pdf, docx ..)
      - 모든 sub module 설치: `pip install unstructured[all-docs]`
      - https://docs.unstructured.io/open-source/installation/full-installation
  - `pip install langchain-unstructured -qU`

### Directory 내의 문서파일들 로딩
- DirectoryLoader 이용

# Chunking (문서 분할)

![rag_split](figures/rag_split.png)

- Load 한 문서를 지정한 기준의 덩어리(chunk)로 나누는 작업을 진행한다.

## 나누는 이유
1. **임베딩 모델의 컨텍스트 길이 제한**
    - 대부분의 언어 모델은 한 번에 처리할 수 있는 토큰 수에 제한이 있다. 전체 문서를 통째로 입력하면 이 제한을 초과할 수 있어 처리가 불가능해진다.
2. **검색 정확도 향상**
    - 큰 문서 전체보다는 특정 주제나 내용을 다루는 작은 chunk가 사용자 질문과 더 정확하게 매칭된다. 예를 들어, 100페이지 매뉴얼에서 특정 기능에 대한 질문이 있을 때, 해당 기능을 설명하는 몇 개의 문단만 검색되는 것이 더 효과적이다.
    - 사용자 질문에 대해 문서의 모든 내용이 다 관련있는 것은 아니다. Chunking을 통해 가장 관련성 높은 부분만 선별적으로 활용할 수 있어 답변의 품질이 향상된다.
    - 전체 문서에는 질문과 무관한 내용들이 많이 포함되어 있어 모델이 혼란을 겪을 수 있다. 적절한 크기의 chunk는 이런 노이즈를 줄여준다.
3. **계산 효율성**
    - 벡터 유사도 계산, 임베딩 생성 등의 작업이 작은 chunk 단위로 수행될 때 더 빠르고 효율적이다. 메모리 사용량도 줄일 수 있다.

## 주요 Spliter
- https://api.python.langchain.com/en/latest/text_splitters_api_reference.html

### CharacterTextSplitter
가장  기본적인 Text spliter
- 한개의 구분자를 기준으로 분리한다. (default: "\n\n")
    - 분리된 조각이 chunk size 보다 작으면 다음 조각과 합칠 수 있다.
        - 합쳤을때 chuck_size 보다 크면 안 합친다. chuck_size 이내면 합친다.
    - 나누는 기준은 구분자이기 때문에 chunk_size 보다 글자수가 많을 수 있다.
- chunk size: 분리된 문서(chunk) 글자수 이내에서 분리되도록 한다.
    -  구분자를 기준으로 분리한다. 구분자를 기준으로 분리한 문서 조각이 chunk size 보다 크더라도 그대로 유지한다. 즉 chunk_size가 우선이 아니라 **seperator** 가 우선이다.
- 주요 파라미터
    - chunk_size: 각 조각의 최대 길이를 지정.
    - seperator: 구분 문자열을 지정. (default: '\n\n')
- CharacterTextSplitter는 단순 스플리터로 overlap기능을 지원하지는 않는다. 단 seperator가 빈문자열("") 일 경우에는 overlap 기능을 지원한다. overlap이란 각 이전 청크의 뒷부분의 문자열을 앞에 붙여 문맥을 유지하는 것을 말한다.
  
### RecursiveCharacterTextSplitter
- RecursiveCharacterTextSplitter는 **긴 텍스트를 지정된 최대 길이(chunk_size) 이하로 나누는 데 효과적인 텍스트 분할기**(splitter)이다.
- 여러 **구분자(separators)를 순차적으로 적용**하여, 가능한 한 자연스러운 문단/문장/단어 단위로 분할하고, 최종적으로는 크기 제한을 만족시킨다.
- 분할 기준 문자
    1. 두 개의 줄바꿈 문자 ("\n\n")
    2. 한 개의 줄바꿈 문자 ("\n")
    3. 공백 문자 (" ")
    4. 빈 문자열 ("")
- 작동 방식
    1. 먼저 가장 높은 우선순위의 구분자("\n\n")로 분할을 시도한다.
    2. 분할된 조각 중 **chunk_size를 초과하는 조각**에 대해 다음 우선순위 구분자("\n" → " " → "")로 재귀적으로 재분할한다.
    3. 이 과정을 통해 모든 조각(chunk)이 chunk_size를 초과하지 않도록 만든다.  
- 주요 파라미터
    - chunk_size: 각 조각의 최대 길이를 지정.
    - chunk_overlap: 연속된 청크들 간의 겹치는 문자 수를 설정. 새로운 청크 생성 시 이전 청크의 마지막 부분에서 지정된 수만큼의 문자를 가져와서 새 청크의 앞부분에 포함시켜, 청크 경계에서 문맥의 연속성을 유지한다.
      - 구분자에 의해 청크가 나눠지면 정상적인 분리이므로 overlap이 적용되지 않는다.
      - 정상적 구분자로 나눌 수 없어 chunk_size에 맞춰 잘라진 경우 문맥의 연결성을 위애 overlap을 적용한다.
    - separators(list): 구분자를 지정한다. 지정하면 기본 구분자가 지정한 것으로 변경된다.

#### 메소드
- `split_documents(Iterable[Document]) : List[Document]`
    - Document 목록을 받아 split 처리한다.
- `split_text(str) : List[str]`
    - string text를 받아서 split 처리한다. 

In [ ]:
text = """가각간갇갈갉갊감갑값갓갔강갖갗같갚갛개객갠갤갬갭갯갰

aadlskfjadklsfjakldfjadklsjadfskl갸갹갼걀걋걍걔걘걜거걱건걷걸걺검겁것겉겊겋게겐

띱띳띵라락란랄람랍랏랐

랑랒랖랗래랙랜랠램랩랫랬랭랴략랸럇량러럭런럴럼럽럿렀렁렇레렉렌렐렘렙렛렝나낙낚ASDFFGHJJKKLLLQWE

멨멩며 

멱면멸몃몄명몇몌모목몫몬몰몲몸몹못몽뫄뫈뫘뫙뫼묀묄묍묏묑묘묜묠묩묫무묵묶문묻물묽묾뭄뭅뭇뭉뭍뭏ABCDEFGHIJ"""

## Token 수 기준으로 나누기

- LLM 언어 모델들은 입력 토큰 수 제한이 있어서 요청시 제한 토큰수 이상의 프롬프트는 전송할 수 없다.
- 따라서 텍스트를 chunk로 분할할 때는 글자수 보다 **토큰 수를 기준으로 크기를 지정하는 것**이 좋다.  
- 토큰기반 분할은 텍스트의 의미를 유지하면서 분할하는 방식이므로 문자 기반 분할과 같이 단어가 중간잘리는 것들을 방지할 수 있다. 
- 토큰 수 계산할 때는 사용하는 언어 모델에 사용된 것과 동일한 tokenizer를 사용하는 것이 좋다.
  - 예를 들어 OpenAI의 GPT 모델을 사용할 경우 tiktoken 라이브러리를 활용하여 토큰 수를 정확하게 계산할 수 있다.

### [tiktoken](https://github.com/openai/tiktoken) tokenizer 기반 분할
- OpenAI에서 GPT 모델을 학습할 때 사용한 `BPE` 방식의 tokenizer. **OpenAI 언어모델을 사용할 경우 이것을 사용하는 것이 좀 더 정확하게  토큰dmf 계산할 수 있다.**
- Splitter.from_tiktoken_encoder() 메소드를 이용해 생성
  - `RecursiveCharacterTextSplitter.from_tiktoken_encoder()`
  - `CharacterTextSplitter.from_tiktoken_encoder()`
- 파라미터
  - encode_name: 인코딩 방식(토큰화 규칙)을 지정. OpenAI는 GPT 모델들 마다 다른 방식을 사용했다. 그래서 사용하려는 모델에 맞는 인코딩 방식을 지정해야 한다.
    - `cl100k_base`: GPT-4 및 GPT-3.5-Turbo 모델에서 사용된 방식.
    - `r50k_base:` GPT-3 모델에서 사용된 방식 
  - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- tiktoken 설치
  - `pip install tiktoken`

### HuggingFace Tokenizer
- HuggingFace 모델을 사용할 경우 그 모델이 사용한 tokenizer를 이용해 토큰 기반으로 분할 한다.
  - 다른 tokenizer를 이용해 분할 할 경우 토큰 수 계산이 다르게 될 수있다.
- `from_huggingface_tokenizer()` 메소드를 이용.
  - 파라미터
    - tokenizer: HuggingFace tokenizer 객체
    - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- `transformers` 라이브러리를 설치해야 한다.
  - `pip install transformers` 